In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.regularizers import l2
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import regularizers
from tensorflow.python.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import MultiHeadAttention
from keras.layers import Input
from keras.layers import Concatenate,GlobalAveragePooling1D
from keras.layers import Masking, LSTM, TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import smote_variants as sv

In [2]:
data1 = pd.read_csv("dataset/AfterDeletejedit40.csv")
(row, cell) = data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [3]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row,cell-1,1)
X_train, y_train = X_res, y_res

In [4]:
data1 = pd.read_csv("dataset/AfterDeletejedit41.csv")
(row1, cell1) = data1.values.shape
data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [5]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row1,cell-1,1)
X_test, y_test = X_res, y_res

In [6]:
# 转换标签为独热编码
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((419, 20, 1), (283, 20, 1), (419, 2), (283, 2))

In [7]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
threshold = 0.5  

def mcc_metric(y_true, y_pred):
    predicted = tf.cast(tf.greater(y_pred, threshold), tf.float32)
    true_pos = tf.math.count_nonzero(predicted * y_true)
    true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
    false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
    false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
    x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
        * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
    return tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)


In [8]:
num_classes = 2

def baseline_model():
    # Define input layer
    inputs = Input(shape=(cell-1, 1))
    
    # Convolutional layer
    conv = Conv1D(filters=12, kernel_size=4, activation='relu', kernel_regularizer='l2')(inputs)
    
    # Max pooling layer
    pool = MaxPooling1D(pool_size=2)(conv)
    
    # Multi-head attention layer
    multi_attention = MultiHeadAttention(num_heads=2, key_dim=2)(pool, pool)  # Applying multi-head attention to the pooled features
    
    # Flatten layer
    flatten = Flatten()(multi_attention)
    
    # Fully connected layer
    fc = Dense(21, activation='relu', kernel_regularizer='l2')(flatten)
    
    # Output layer
    outputs = Dense(num_classes, activation='sigmoid', kernel_regularizer='l2')(fc)
    
    # Creating model
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, recall_m, mcc_metric])
    
    return model


In [9]:

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [10]:
# build the model
model = baseline_model()
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=20,verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[3]*100))


Epoch 1/16
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7888 - f1_m: 0.5968 - loss: 8.9986 - mcc_metric: 0.3520 - recall_m: 0.5023 - val_accuracy: 0.7719 - val_f1_m: 0.6824 - val_loss: 1.5573 - val_mcc_metric: 0.4022 - val_recall_m: 0.6461
Epoch 2/16
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7943 - f1_m: 0.7548 - loss: 2.0259 - mcc_metric: 0.5149 - recall_m: 0.7603 - val_accuracy: 0.8421 - val_f1_m: 0.7911 - val_loss: 1.1863 - val_mcc_metric: 0.5740 - val_recall_m: 0.8049
Epoch 3/16
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8100 - f1_m: 0.7717 - loss: 2.1828 - mcc_metric: 0.5289 - recall_m: 0.8067 - val_accuracy: 0.8070 - val_f1_m: 0.7542 - val_loss: 1.5639 - val_mcc_metric: 0.5853 - val_recall_m: 0.6765
Epoch 4/16
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8617 - f1_m: 0.8185 - loss: 1.8328 - mcc_metric: 0.6616 - recall_m: 0.7843 - val_accuracy: 0.8596 - val_f1_m: 0.8383 - val_loss: 1.1135 - val_mcc_metric: 0.6635 - val_recall_m: 0.8745
Epoch 5

In [13]:
print(" Loss               Accuracy            F1 score             Recall              Mcc         ")
print(scores)


 Loss               Accuracy            F1 score             Recall              Mcc         
[0.8176923990249634, 0.8070175647735596, 0.8192306756973267, 0.8418749570846558, 0.6292499899864197]
